In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir, makedirs
from os.path import join, basename, isdir

medir = lambda x: [ y for y in dir(x) if y[0]!='_']

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
join_columns_levels = lambda df, sep=' ': df.set_axis([sep.join(map(str,columns)) for columns in df.columns], axis=1).rename_axis(sep.join(df.columns.names), axis=1)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

import numpy as np
from epiweeks import Week, Year
from datetime import date, timedelta

In [ ]:
from bulletin import hoje, default_output
from bulletin.systems import Notifica
from bulletin.utils.xls_ import fit_cols, format_cols

default_output = join(default_output,"testes")

if not isdir(default_output):
    makedirs(default_output)

In [ ]:
positividade_labels = { 2: 'Casos Confirmados',3: 'Casos Descartados',4: 'Total', 5: 'Positividade' }
rs_labels = { k:f"{k}ª Regional" for k in range(1,23) }
mes_labels = {key+1:value for key,value in enumerate(['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro'])}

In [ ]:
testes_enviados_rs = pd.read_csv('testes_enviados.csv',index_col='rs')
testes_enviados_rs.columns = pd.MultiIndex.from_tuples([(2021,6),(2021,10),(2021,11),(2022,1),(2022,2)])
testes_enviados_rs

In [ ]:
nt = Notifica()
nt.load('notifica',compress=False)
nt.df = nt.df[['id','cnes_unidade_notifica','ibge_unidade_notifica','exame','metodo','data_notificacao','classificacao_final']]

In [ ]:
nt.df

In [ ]:
nt.df.shape

In [ ]:
nt.df = nt.df.drop(index=nt.df.loc[nt.df['data_notificacao'].isna()].index)

In [ ]:
nt.df = nt.df.join(nt.tables['municipios'].set_index('ibge'), on='ibge_unidade_notifica')
nt.df = nt.df.drop(index=nt.df.loc[nt.df['uf']!='PR'].index)

In [ ]:
cnes = pd.read_csv('cnes.csv')
# cnes['estabelecimento_nome_fantasia'] = cnes['cnes'].apply(lambda x: ' '.join(x.split(' ')[1:]) )
cnes.index = cnes['cnes'].apply(lambda x: x.split(' ')[0] )
cnes['doc'] = cnes['cnes'].apply(lambda x: x.split(' ')[1] )
cnes['value'] = cnes['cnes'].apply(lambda x: ''.join(filter(lambda y: str.isnumeric(y),x.split(' ')[2])) )
cnes['name'] = cnes['cnes'].apply(lambda x: ''.join(filter(lambda y: (str.isalpha(y)) or (y==' '),' '.join(x.split(' ')[2:]))) )
del cnes['cnes']
cnes

In [ ]:
cnes['eh_sus'] = 0
cnes.loc[cnes['value'].apply(lambda x: sum(map(int,x)))==0,'eh_sus'] = 1
cnes.groupby('eh_sus')[['doc']].count()

In [ ]:
nt.df['eh_sus'] = 0
nt.df.loc[nt.df['cnes_unidade_notifica'].isin(cnes.loc[cnes['eh_sus']==1].index),'eh_sus'] = 1

In [ ]:
nt.df.groupby('eh_sus')[['id']].count()

In [ ]:
nt.df = nt.df.drop(index=nt.df.loc[nt.df['eh_sus']==0].index)

In [ ]:
nt.df.loc[nt.df['classificacao_final']!=2,'classificacao_final'] = 3

In [ ]:
# nt.df['mes'] = nt.df['data_notificacao'].apply(lambda x: Week.fromdate(x).week)
# nt.df['ano'] = nt.df['data_notificacao'].apply(lambda x: Week.fromdate(x).year)

nt.df['mes'] = nt.df['data_notificacao'].dt.month
nt.df['ano'] = nt.df['data_notificacao'].dt.year

In [ ]:
metodos = nt.tables['metodo']
metodos

In [ ]:
exames_rt = [1,2,27,181]
exames_tr = [167,188,189,190,191]

exames = nt.tables['exame'].set_index('id').sort_index()
exames.loc[exames_rt + exames_tr].sort_index()

##  Testes rápidos

In [ ]:
tr = nt.df.loc[(nt.df['metodo']==2) & nt.df['exame'].isin(exames_tr)]
# tr = nt.df.loc[(nt.df['metodo']==2)]

### PR Total

In [ ]:
# tr_PR_total = tr.groupby(['ano','classificacao_final'])[['id']].count().unstack('classificacao_final').fillna(0).astype(int).droplevel(axis=1,level=0)
tr_PR_total = tr.groupby(['classificacao_final'])[['id']].count().T.rename(index={'id':'Paraná'})
tr_PR_total[4] = tr_PR_total.sum(axis=1)
tr_PR_total[5] = tr_PR_total[2] / tr_PR_total[4]
tr_PR_total = tr_PR_total.sort_index(axis=1)
tr_PR_total = tr_PR_total.rename(columns=positividade_labels)
tr_PR_total

### Paraná por SE

In [ ]:
tr_PR_se = tr.groupby(['ano','mes','classificacao_final'])[['id']].count().unstack('classificacao_final').fillna(0).astype(int).droplevel(axis=1,level=0)
tr_PR_se[4] = tr_PR_se.sum(axis=1)
tr_PR_se[5] = tr_PR_se[2] / tr_PR_se[4]
tr_PR_se = tr_PR_se.sort_index(axis=1)
tr_PR_se = tr_PR_se.rename(columns=positividade_labels)
tr_PR_se.tail()

### Regionais Total

In [ ]:
tr_RS_total = tr.groupby(['rs','classificacao_final'])[['id']].count().unstack(['classificacao_final']).fillna(0).astype(int).droplevel(axis=1,level=0).sort_index(axis=1, level='rs')
tr_RS_total_TOTAL = tr_RS_total.sum(axis=1)
tr_RS_total_CONF = tr_RS_total.loc[:,2]
tr_RS_total_DESC = tr_RS_total.loc[:,3]
tr_RS_total_POSIT = tr_RS_total_CONF / tr_RS_total_TOTAL.values
tr_RS_total = pd.concat({2: tr_RS_total_CONF, 3: tr_RS_total_DESC, 4: tr_RS_total_TOTAL, 5: tr_RS_total_POSIT}, axis=1, names=['classificacao_final']).sort_index(axis=1,level=['classificacao_final'])
tr_RS_total = tr_RS_total.rename(columns=positividade_labels).rename(index=rs_labels)
tr_RS_total

### Regionais por SE

In [ ]:
tr_RS_se = tr.groupby(['ano','mes','rs','classificacao_final'])[['id']].count().unstack(['rs','classificacao_final']).fillna(0).astype(int).droplevel(axis=1,level=0).sort_index(axis=1, level='rs')
tr_RS_se_TOTAL = tr_RS_se.groupby(level='rs', axis=1).sum()
tr_RS_se_CONF = tr_RS_se.xs(2,level='classificacao_final',axis=1)
tr_RS_se_DESC = tr_RS_se.xs(3,level='classificacao_final',axis=1)
tr_RS_se_POSIT = tr_RS_se_CONF / tr_RS_se_TOTAL.values
tr_RS_se = pd.concat({2: tr_RS_se_CONF, 3: tr_RS_se_DESC, 4: tr_RS_se_TOTAL, 5: tr_RS_se_POSIT}, axis=1, names=['classificacao_final']).reorder_levels(['rs','classificacao_final'],axis=1).sort_index(axis=1,level=['rs','classificacao_final'])
tr_RS_se = tr_RS_se.rename(columns=positividade_labels, level='classificacao_final').rename(columns=rs_labels, level='rs')
tr_RS_se.tail()

In [ ]:
testes_enviados_usados = pd.concat({'enviados': testes_enviados_rs, 'usados': tr_RS_se_TOTAL.T}, names=['status','ano','mes'], axis=1).reorder_levels(['ano','mes','status'],axis=1).sort_index(axis=1,level=['ano','mes','status'])
total_testes_enviados_usados = testes_enviados_usados.groupby(level=['status'],axis=1).sum()
total_testes_enviados_usados['% usados'] = (total_testes_enviados_usados['usados'] / total_testes_enviados_usados['enviados']).apply(lambda x: f"{100*x:.0f}%")
total_testes_enviados_usados['restantes'] = total_testes_enviados_usados['enviados'] - total_testes_enviados_usados['usados']
total_testes_enviados_usados['% restantes'] = (total_testes_enviados_usados['restantes'] / total_testes_enviados_usados['enviados']).apply(lambda x: f"{100*x:.0f}%")
# total_testes_enviados_usados.sort_values('restantes',ascending=True).to_excel('t.xlsx')
total_testes_enviados_usados

### Municipios Totais

In [ ]:
tr_MUN_total = tr.groupby(['rs','ibge_unidade_notifica','municipio','classificacao_final'])[['id']].count().unstack(['classificacao_final']).fillna(0).astype(int).droplevel(axis=1,level=0).sort_index(axis=1, level='rs')
tr_MUN_total[4] = tr_MUN_total.sum(1)
tr_MUN_total[5] = tr_MUN_total[2] / tr_MUN_total[4]
tr_MUN_total = tr_MUN_total.rename(columns=positividade_labels, level='classificacao_final').rename(index=rs_labels, level='rs')
tr_MUN_total = tr_MUN_total.droplevel(level='ibge_unidade_notifica',axis=0)
tr_MUN_total

### Municipios por SE

In [ ]:
tr_MUN_se =  tr.groupby(['ano','mes','rs','ibge_unidade_notifica','municipio','classificacao_final'])[['id']].count().unstack(['rs','ibge_unidade_notifica','municipio','classificacao_final']).fillna(0).astype(int).droplevel(axis=1,level=0).sort_index(axis=1, level='rs')
tr_MUN_se_TOTAL = tr_MUN_se.groupby(level=['rs','ibge_unidade_notifica','municipio'], axis=1).sum()
tr_MUN_se_CONF = tr_MUN_se.xs(2,level='classificacao_final',axis=1)
tr_MUN_se_DESC = tr_MUN_se.xs(3,level='classificacao_final',axis=1)
tr_MUN_se_POSIT = tr_MUN_se_CONF / tr_MUN_se_TOTAL.values
tr_MUN_se = pd.concat({2: tr_MUN_se_CONF, 3: tr_MUN_se_DESC, 4: tr_MUN_se_TOTAL, 5: tr_MUN_se_POSIT}, axis=1, names=['classificacao_final']).reorder_levels(['rs','ibge_unidade_notifica','municipio','classificacao_final'],axis=1).sort_index(axis=1,level=['rs','ibge_unidade_notifica','municipio','classificacao_final'])
tr_MUN_se = tr_MUN_se.rename(columns=positividade_labels, level='classificacao_final').rename(columns=rs_labels, level='rs')
tr_MUN_se = tr_MUN_se.droplevel(level='ibge_unidade_notifica',axis=1)
tr_MUN_se.tail()

In [ ]:
writer = pd.ExcelWriter(join(default_output,"panorama_teste_rapido.xlsx"))

# Add some cell formats.
number = writer.book.add_format({'num_format': '#,##0'})
percent = writer.book.add_format({'num_format': '0%'})

col_formats = {'Casos Confirmados': number, 'Casos Descartados': number, 'Total': number, 'Positividade':percent}

tr_PR_total.to_excel(writer,'tr_PR_total',merge_cells=False)
format_cols(writer,tr_PR_total,'tr_PR_total',col_formats)

tr_PR_se.to_excel(writer,'tr_PR_se',merge_cells=False)
format_cols(writer,tr_PR_se,'tr_PR_se',col_formats)

tr_RS_total.to_excel(writer,'tr_RS_total',merge_cells=False)
format_cols(writer,tr_RS_total,'tr_RS_total',col_formats)

tr_RS_se.to_excel(writer,'tr_RS_se',merge_cells=False)
format_cols(writer,tr_RS_se,'tr_RS_se',col_formats)

tr_MUN_total.to_excel(writer,'tr_MUN_total',merge_cells=False)
format_cols(writer,tr_MUN_total,'tr_MUN_total',col_formats)

tr_MUN_se.to_excel(writer,'tr_MUN_se',merge_cells=False)
format_cols(writer,tr_MUN_se,'tr_MUN_se',col_formats)

writer.save()

##  Exame RT-PCR

In [ ]:
rt_pcr = nt.df.loc[(nt.df['metodo']==1) & nt.df['exame'].isin(exames_rt)]
# rt_pcr = nt.df.loc[(nt.df['metodo']==1)]

### PR Total

In [ ]:
# rt_pcr_PR_total = rt_pcr.groupby(['ano','classificacao_final'])[['id']].count().unstack('classificacao_final').fillna(0).astype(int).droplevel(axis=1,level=0)
rt_pcr_PR_total = rt_pcr.groupby(['classificacao_final'])[['id']].count().T.rename(index={'id':'Paraná'})
rt_pcr_PR_total[4] = rt_pcr_PR_total.sum(axis=1)
rt_pcr_PR_total[5] = rt_pcr_PR_total[2] / rt_pcr_PR_total[4]
rt_pcr_PR_total = rt_pcr_PR_total.sort_index(axis=1)
rt_pcr_PR_total = rt_pcr_PR_total.rename(columns=positividade_labels)
rt_pcr_PR_total

### Paraná por SE

In [ ]:
rt_pcr_PR_se = rt_pcr.groupby(['ano','mes','classificacao_final'])[['id']].count().unstack('classificacao_final').fillna(0).astype(int).droplevel(axis=1,level=0)
rt_pcr_PR_se[4] = rt_pcr_PR_se.sum(axis=1)
rt_pcr_PR_se[5] = rt_pcr_PR_se[2] / rt_pcr_PR_se[4]
rt_pcr_PR_se = rt_pcr_PR_se.sort_index(axis=1)
rt_pcr_PR_se = rt_pcr_PR_se.rename(columns=positividade_labels)
rt_pcr_PR_se.tail()

### Regionais Total

In [ ]:
rt_pcr_RS_total = rt_pcr.groupby(['rs','classificacao_final'])[['id']].count().unstack(['classificacao_final']).fillna(0).astype(int).droplevel(axis=1,level=0).sort_index(axis=1, level='rs')
rt_pcr_RS_total_TOTAL = rt_pcr_RS_total.sum(axis=1)
rt_pcr_RS_total_CONF = rt_pcr_RS_total.loc[:,2]
rt_pcr_RS_total_DESC = rt_pcr_RS_total.loc[:,3]
rt_pcr_RS_total_POSIT = rt_pcr_RS_total_CONF / rt_pcr_RS_total_TOTAL.values
rt_pcr_RS_total = pd.concat({2: rt_pcr_RS_total_CONF, 3: rt_pcr_RS_total_DESC, 4: rt_pcr_RS_total_TOTAL, 5: rt_pcr_RS_total_POSIT}, axis=1, names=['classificacao_final']).sort_index(axis=1,level=['classificacao_final'])
rt_pcr_RS_total = rt_pcr_RS_total.rename(columns=positividade_labels).rename(index=rs_labels)
rt_pcr_RS_total

### Regionais por SE

In [ ]:
rt_pcr_RS_se = rt_pcr.groupby(['ano','mes','rs','classificacao_final'])[['id']].count().unstack(['rs','classificacao_final']).fillna(0).astype(int).droplevel(axis=1,level=0).sort_index(axis=1, level='rs')
rt_pcr_RS_se_TOTAL = rt_pcr_RS_se.groupby(level='rs', axis=1).sum()
rt_pcr_RS_se_CONF = rt_pcr_RS_se.xs(2,level='classificacao_final',axis=1)
rt_pcr_RS_se_DESC = rt_pcr_RS_se.xs(3,level='classificacao_final',axis=1)
rt_pcr_RS_se_POSIT = rt_pcr_RS_se_CONF / rt_pcr_RS_se_TOTAL.values
rt_pcr_RS_se = pd.concat({2: rt_pcr_RS_se_CONF, 3: rt_pcr_RS_se_DESC, 4: rt_pcr_RS_se_TOTAL, 5: rt_pcr_RS_se_POSIT}, axis=1, names=['classificacao_final']).reorder_levels(['rs','classificacao_final'],axis=1).sort_index(axis=1,level=['rs','classificacao_final'])
rt_pcr_RS_se = rt_pcr_RS_se.rename(columns=positividade_labels, level='classificacao_final').rename(columns=rs_labels, level='rs')
rt_pcr_RS_se.tail()

### Municipios Totais

In [ ]:
rt_pcr_MUN_total = rt_pcr.groupby(['rs','ibge_unidade_notifica','municipio','classificacao_final'])[['id']].count().unstack(['classificacao_final']).fillna(0).astype(int).droplevel(axis=1,level=0).sort_index(axis=1, level='rs')
rt_pcr_MUN_total[4] = rt_pcr_MUN_total.sum(1)
rt_pcr_MUN_total[5] = rt_pcr_MUN_total[2] / rt_pcr_MUN_total[4]
rt_pcr_MUN_total = rt_pcr_MUN_total.rename(columns=positividade_labels, level='classificacao_final').rename(index=rs_labels, level='rs')
rt_pcr_MUN_total = rt_pcr_MUN_total.droplevel(level='ibge_unidade_notifica',axis=0)
rt_pcr_MUN_total

### Municipios por SE

In [ ]:
rt_pcr_MUN_se =  rt_pcr.groupby(['ano','mes','rs','ibge_unidade_notifica','municipio','classificacao_final'])[['id']].count().unstack(['rs','ibge_unidade_notifica','municipio','classificacao_final']).fillna(0).astype(int).droplevel(axis=1,level=0).sort_index(axis=1, level='rs')
rt_pcr_MUN_se_TOTAL = rt_pcr_MUN_se.groupby(level=['rs','ibge_unidade_notifica','municipio'], axis=1).sum()
rt_pcr_MUN_se_CONF = rt_pcr_MUN_se.xs(2,level='classificacao_final',axis=1)
rt_pcr_MUN_se_DESC = rt_pcr_MUN_se.xs(3,level='classificacao_final',axis=1)
rt_pcr_MUN_se_POSIT = rt_pcr_MUN_se_CONF / rt_pcr_MUN_se_TOTAL.values
rt_pcr_MUN_se = pd.concat({2: rt_pcr_MUN_se_CONF, 3: rt_pcr_MUN_se_DESC, 4: rt_pcr_MUN_se_TOTAL, 5: rt_pcr_MUN_se_POSIT}, axis=1, names=['classificacao_final']).reorder_levels(['rs','ibge_unidade_notifica','municipio','classificacao_final'],axis=1).sort_index(axis=1,level=['rs','ibge_unidade_notifica','municipio','classificacao_final'])
rt_pcr_MUN_se = rt_pcr_MUN_se.rename(columns=positividade_labels, level='classificacao_final').rename(columns=rs_labels, level='rs')
rt_pcr_MUN_se = rt_pcr_MUN_se.droplevel(level='ibge_unidade_notifica',axis=1)
rt_pcr_MUN_se.tail()

In [ ]:
writer = pd.ExcelWriter(join(default_output,"panorama_rt_pcr.xlsx"))

# Add some cell formats.
number = writer.book.add_format({'num_format': '#,##0'})
percent = writer.book.add_format({'num_format': '0%'})

col_formats = {'Casos Confirmados': number, 'Casos Descartados': number, 'Total': number, 'Positividade':percent}

rt_pcr_PR_total.to_excel(writer,'rt_pcr_PR_total',merge_cells=False)
format_cols(writer,rt_pcr_PR_total,'rt_pcr_PR_total',col_formats)

rt_pcr_PR_se.to_excel(writer,'rt_pcr_PR_se',merge_cells=False)
format_cols(writer,rt_pcr_PR_se,'rt_pcr_PR_se',col_formats)

rt_pcr_RS_total.to_excel(writer,'rt_pcr_RS_total',merge_cells=False)
format_cols(writer,rt_pcr_RS_total,'rt_pcr_RS_total',col_formats)

rt_pcr_RS_se.to_excel(writer,'rt_pcr_RS_se',merge_cells=False)
format_cols(writer,rt_pcr_RS_se,'rt_pcr_RS_se',col_formats)

rt_pcr_MUN_total.to_excel(writer,'rt_pcr_MUN_total',merge_cells=False)
format_cols(writer,rt_pcr_MUN_total,'rt_pcr_MUN_total',col_formats)

rt_pcr_MUN_se.to_excel(writer,'rt_pcr_MUN_se',merge_cells=False)
format_cols(writer,rt_pcr_MUN_se,'rt_pcr_MUN_se',col_formats)

writer.save()